**1. Import Libraries**

In [29]:
import pyodbc
import pandas as pd 
import numpy as np 
import configparser
import traceback


#Display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

**2. Load Database Configuration**

In [30]:
config = configparser.ConfigParser()
config_path = '../config.ini'

try:
    config.read(config_path)
    db_config = config['database']
    
    server = db_config['server']
    database = db_config['database']
    driver = db_config['driver']
    trusted_connection = db_config.getboolean('trusted_connection', fallback=False)
    
    conn_str_parts = [
        f"Driver={{{driver}}}",
        f"Server={{{server}}}",
        f"Database={{{database}}}"
    ]
    
    # ... (inside the try block for config parsing)
    trust_server_cert_str = db_config.get('trust_server_certificate', 'no') # Default to 'no'
    trust_server_certificate = configparser.ConfigParser.BOOLEAN_STATES.get(trust_server_cert_str.lower(), False)
    print(f"Trust Server Certificate: {trust_server_certificate} (from string '{trust_server_cert_str}')")

    if trust_server_certificate:
        conn_str_parts.append("TrustServerCertificate=yes")
    # ...
    # Debug print the final connection string (mask password if necessary)
    log_conn_str_parts = list(conn_str_parts)
    for i, part in enumerate(log_conn_str_parts):
        if part.lower().startswith("pwd="):
            log_conn_str_parts[i] = "PWD=********"
        print(f"Final Connection String Being Used (Password Masked): {';'.join(log_conn_str_parts)}")
    else:
        uid = db_config.get('uid')
        pwd = db_config.get('pwd')
        if not uid or not pwd:
            raise ValueError("UID and PWD must be provided in config.ini if Trusted_Connection = 'no'")
        conn_str_parts.append(f"UID={uid}")
        conn_str_parts.append(f"PWD={pwd}")
    
    if db_config.getboolean('trust_server_certificate', fallback=False):# Add more from your schema
        conn_str_parts.append("TrustedServerCertificate=yes")
    
    print(f"DEBUG: conn_str_parts before join: {conn_str_parts}")
    conn_str = ";".join(conn_str_parts)
    print("Connection string configured (password hidden if applicable).")

except Exception as e:
    print(f"Error loading configuration or Building connection string: {e}")
    conn_str = None


Trust Server Certificate: True (from string 'yes')
Final Connection String Being Used (Password Masked): Driver={ODBC Driver 18 for SQL Server};Server={localhost};Database={QuickPesaDB};TrustServerCertificate=yes
Final Connection String Being Used (Password Masked): Driver={ODBC Driver 18 for SQL Server};Server={localhost};Database={QuickPesaDB};TrustServerCertificate=yes
Final Connection String Being Used (Password Masked): Driver={ODBC Driver 18 for SQL Server};Server={localhost};Database={QuickPesaDB};TrustServerCertificate=yes
Final Connection String Being Used (Password Masked): Driver={ODBC Driver 18 for SQL Server};Server={localhost};Database={QuickPesaDB};TrustServerCertificate=yes
DEBUG: conn_str_parts before join: ['Driver={ODBC Driver 18 for SQL Server}', 'Server={localhost}', 'Database={QuickPesaDB}', 'TrustServerCertificate=yes', 'UID=sa', 'PWD=BLOOMberg411**', 'TrustedServerCertificate=yes']
Connection string configured (password hidden if applicable).


**3. Execute query and load into dataframe**

In [31]:
sql_query = ""
try:
    with open('../sql_queries/main_data_extraction.sql', 'r') as file:
        sql_query = file.read()
except FileNotFoundError:
    print("Error: SQL query file not found. Make sure 'main_data_extraction.sql' is in the 'sql_queries' directory.")
    sql_query = None
    
df = None
if conn_str and sql_query:
    try:
        conn = pyodbc.connect(conn_str)
        print("Database connection successful")
        
        df = pd.read_sql(sql_query, conn)
        conn.close()
        print(f"Data extracted successfully. Shape: {df.shape}")
        print(df.head())
    
    except pyodbc.Error as ec:
        sqlstate = ec.args[0]
        print(f"Database execution error: {sqlstate}")
        print(ec)
    except Exception as e:
        print(f"An error occurred during pd.read_sql or connection closing: {type(e).__name__} - {e}")
        traceback.print_exc()

else:
    if not conn_str:
        print("Database connection string not configured. Skipping data extraction.")
    if not sql_query:
        print("SQL query is empty or file not read. Skipping data extraction.")
        
if df is not None:
    try: 
        df.to_csv('../data/raw_loan_data.csv', index=False)
        print("Raw data saved to data/raw_loan_data.csv")
    except Exception as e:
        print(f"Error saving raw data: {e}")
else:
    print("DataFrame is None. Cannot save.")


Database connection successful


/tmp/ipykernel_9025/1466322030.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


Data extracted successfully. Shape: (65830, 47)
   LoanID  LoanPrincipal  TotalRepayable LoanStatus        DisbursementDate  \
0   23057          400.0          452.67  Defaulted 2024-02-23 06:35:03.777   
1   23061          900.0         1018.50       Paid 2024-02-23 02:33:36.777   
2   23062          800.0          905.33       Paid 2024-02-23 07:20:38.777   
3   23063          300.0          339.50  Defaulted 2024-02-23 04:17:10.777   
4   23064          800.0          905.33       Paid 2024-02-23 00:22:05.777   

                  DueDate  LoanTermActualDays  DaysDelayed  \
0 2024-03-01 06:35:03.777                   7          -26   
1 2024-03-01 02:33:36.777                   7            3   
2 2024-03-01 07:20:38.777                   7           -6   
3 2024-03-01 04:17:10.777                   7          452   
4 2024-03-01 00:22:05.777                   7          -53   

          ApplicationDate  AmountRequested  LoanTermRequestedDays  \
0 2024-02-23 05:23:03.777          

**4. Initial Data Inspection**

In [32]:
if df is not None:
    print("Data Info:")
    df.info()
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nDescriptive Statistics:")
    print(df.describe(include='all'))
else:
    print("Dataframe is not loaded. Cannot perform inspection.")

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65830 entries, 0 to 65829
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   LoanID                       65830 non-null  int64         
 1   LoanPrincipal                65830 non-null  float64       
 2   TotalRepayable               65830 non-null  float64       
 3   LoanStatus                   65830 non-null  object        
 4   DisbursementDate             65830 non-null  datetime64[ns]
 5   DueDate                      65830 non-null  datetime64[ns]
 6   LoanTermActualDays           65830 non-null  int64         
 7   DaysDelayed                  65830 non-null  int64         
 8   ApplicationDate              65830 non-null  datetime64[ns]
 9   AmountRequested              65830 non-null  float64       
 10  LoanTermRequestedDays        65830 non-null  int64         
 11  LoanPurpose                  6

**5. Define Target Variable**

In [33]:
if df is not None:
    # Convert 'Loan Status' to a binary column 'IsDefault' with 'Defaulted' as 1 and 'Paid' as 0
    df['IsDefault'] = df['LoanStatus'].apply(lambda x: 1 if x == 'Defaulted' else 0)
    print("\nValue counts for IsDefault:")
    print(df['IsDefault'].value_counts(normalize=True))


Value counts for IsDefault:
IsDefault
1    0.565487
0    0.434513
Name: proportion, dtype: float64


**6. Handling Missing Values**

In [34]:
if df is not None:
    if 'AvgDepositLast6Months' in df.columns:
        if df['AvgDepositLast6Months'].isnull().any():
            df['AvgDepositLast6Months_ImputedFlag'] = df['AvgDepositLast6Months'].isnull().astype(int)
            df['AvgDepositLast6Months'].fillna(0, inplace=True)
            print(f"\nMissing 'AvgDepositLast6Months' after imputation: {df['AvgDepositLast6Months'].isnull().sum()}")
            print(f"Number of imputations for 'AvgDepositLast6Months': {df['AvgDepositLast6Months_ImputedFlag'].sum()}")
        else:
            print("\nNo missing values found in 'AvgDepositLast6Months'.")
    else:
        print("\nWarning: 'AvgDepositLast6Months' column not found in Dataframe.")
    
    


Missing 'AvgDepositLast6Months' after imputation: 0
Number of imputations for 'AvgDepositLast6Months': 54388


/tmp/ipykernel_9025/1753621373.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AvgDepositLast6Months'].fillna(0, inplace=True)


**7. Outlier Detection and Handling**

In [35]:
# Define columns that should only contain non-negative values
financial_cols = ['MonthlyIncome', 'MonthlyMobileMoneyVolume', 'AmountRequested', 'LoanPrincipal']

print("--- Starting Final Data Sanitization, Imputation, and Outlier Handling ---")

for col in financial_cols:
    if col not in df.columns:
        print(f"\nWarning: Column '{col}' not found in DataFrame. Skipping.")
        continue

    print(f"\n--- Processing column: '{col}' ---")

    # Stage 1: Sanitize Data - Force to numeric and remove invalid negative values
    # -----------------------------------------------------------------
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        print(f"Stage 1: Found {negative_count} invalid negative values. Converting them to NaN.")
        df.loc[df[col] < 0, col] = np.nan
    else:
        print("Stage 1: No negative values found. Data is clean.")

    # Stage 2: Impute Missing Values
    # -----------------------------------------------------------------
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        # Check if there's any valid data left to calculate a median
        if df[col].notna().any():
            # If yes, impute with the median of the valid data
            median_val = df[col].median()
            print(f"Stage 2: Found {missing_count} missing/invalid values. Imputing with calculated median: {median_val:.2f}")
            df[col] = df[col].fillna(median_val)
        else:
            # If no, the entire column was invalid. Fall back to imputing with 0.
            print(f"Stage 2: Entire column contained invalid data. Imputing all {missing_count} values with 0.")
            df[col] = df[col].fillna(0)
    else:
        print("Stage 2: No missing values to impute.")

    # Diagnostic: Describe the data AFTER sanitizing and imputing
    print("\nData description after sanitizing and imputing:")
    print(df[col].describe())

    # Stage 3: Handle Outliers on the clean data
    # -----------------------------------------------------------------
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    # Only perform outlier capping if there is variance in the data (IQR > 0)
    if IQR > 0:
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Enforce a hard minimum of 0 for financial data
        lower_bound = max(0, lower_bound)

        print(f"\nStage 3: Outlier detection results (IQR={IQR:.2f}):")
        print(f"Upper bound: {upper_bound:.2f}, Lower bound: {lower_bound:.2f}")

        # Cap the outliers using .clip() for efficiency
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
        print("Stage 3: Outliers have been capped.")
    else:
        print("\nStage 3: IQR is zero. No outlier capping needed as all values are the same.")

    # Final Diagnostic for the column
    print("\nFinal data description:")
    print(df[col].describe())

print("\n--- Finished all data processing ---")


--- Starting Final Data Sanitization, Imputation, and Outlier Handling ---

--- Processing column: 'MonthlyIncome' ---
Stage 1: No negative values found. Data is clean.
Stage 2: No missing values to impute.

Data description after sanitizing and imputing:
count     65830.000000
mean      59650.961121
std       57807.451711
min        2014.240000
25%       15749.970000
50%       39941.110000
75%       81803.200000
max      229930.010000
Name: MonthlyIncome, dtype: float64

Stage 3: Outlier detection results (IQR=66053.23):
Upper bound: 180883.04, Lower bound: 0.00
Stage 3: Outliers have been capped.

Final data description:
count     65830.000000
mean      58011.445460
std       53759.946929
min        2014.240000
25%       15749.970000
50%       39941.110000
75%       81803.200000
max      180883.045000
Name: MonthlyIncome, dtype: float64

--- Processing column: 'MonthlyMobileMoneyVolume' ---
Stage 1: No negative values found. Data is clean.
Stage 2: No missing values to impute.

Data 

**8. Feature Engineering**

In [36]:
if df is not None:
    #Debt-To-Income Ratio(DTI)
    df['DTI'] = np.where(df['MonthlyIncome'] > 0, df['AmountRequested'] / df['MonthlyIncome'], np.nan)
    df['DTI'].fillna(df['DTI'].median(), inplace=True)
    
    #Loan Amount to Max ELigible Amount Ratio
    df['LoanToMaxEligibleRatio'] = np.where(df['MaxEligibleLoanAmount'] > 0, df['AmountRequested']/ df['MaxEligibleLoanAmount'], np.nan)
    df['LoanToMaxEligibleRatio'].fillna(df['LoanToMaxEligibleRatio'].median(), inplace=True)
    
    # Ratio of monthly income to the requested loan amount
    df['IncomeToLoanRatio'] = np.where(df['AmountRequested'] > 0, df['MonthlyIncome'] / df['AmountRequested'], np.nan)
    
    # Mobile Money Behavior
    # Ratio of total money volume to income, could indicate reliance on mobile money
    df['MobileMoneyTurnoverRatio'] = np.where(df['MonthlyIncome'] > 0, df['MonthlyMobileMoneyVolume'] / df['MonthlyIncome'], np.nan)

    
    # Time since last default (in days, at time of application)
    # Convert date columns to datetime objects first
    df['ApplicationDate'] = pd.to_datetime(df['ApplicationDate'])
    df['PreviousLastDefaultDate'] = pd.to_datetime(df['PreviousLastDefaultDate'])
    df['DaysSinceLastDefault'] = (df['ApplicationDate'] - df['PreviousLastDefaultDate']).dt.days
    df['DaysSinceLastDefault'].fillna(365*10, inplace=True)
    
    #Customer Tenure (in days, at time of application)
    df['CustomerRegistrationDate'] = pd.to_datetime(df['CustomerRegistrationDate'])
    df['CustomerTenureDays'] = (df['ApplicationDate'] - df['CustomerRegistrationDate']).dt.days
    df['CustomerTenureDays'].fillna(0, inplace=True) #if registration date is missing
    
    # Payment History Features
    #Calculate the ratio of on-time vs. defaulted previous loans
    df['OnTimeRepaymentRatio'] = np.where(df['PreviousLoansTaken'] > 0, 
                                     (df['PreviousLoansTaken'] - df['PreviousDefaults']) / df['PreviousLoansTaken'], np.nan)
    
    # Loan History Features
    # Calculate how frequently a customer takes a loan (loans per day of tenure)
    # Avoid division by zero for new customers
    df['LoanFrequency'] = np.where(df['CustomerTenureDays'] > 0, 
                                df['PreviousLoansTaken'] / df['CustomerTenureDays'], np.nan)
    
    
    print("\nEngineered features created. df.head():")
    print(df[['DTI', 'LoanToMaxEligibleRatio', 'IncomeToLoanRatio', 'MobileMoneyTurnoverRatio', 'DaysSinceLastDefault', 'CustomerTenureDays',
               'OnTimeRepaymentRatio', 'LoanFrequency']].head())
   
    
    


Engineered features created. df.head():
        DTI  LoanToMaxEligibleRatio  IncomeToLoanRatio  \
0  0.034278                 0.16000          29.173100   
1  0.018062                 0.90000          55.363656   
2  0.004423                 0.80000         226.093412   
3  0.065103                 0.30000          15.360267   
4  0.021239                 0.13913          47.083462   

   MobileMoneyTurnoverRatio  DaysSinceLastDefault  CustomerTenureDays  \
0                  0.518982                -462.0                 338   
1                  0.476702               44301.0                 566   
2                  0.173705               45136.0                 242   
3                  0.503407               45151.0                 593   
4                  0.437748                -462.0                  91   

   OnTimeRepaymentRatio  LoanFrequency  
0              0.473684       0.112426  
1              0.680851       0.083039  
2              0.615385       0.161157  
3      

/tmp/ipykernel_9025/3805722961.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['DTI'].fillna(df['DTI'].median(), inplace=True)
/tmp/ipykernel_9025/3805722961.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

**9. Save processed Data**

In [37]:
if df is not None:
    df.to_csv('../data/processed_loan_data.csv', index=False)
    print("Processed data saved to /processed_loan_data.csv")

Processed data saved to /processed_loan_data.csv
